## 特定のWalletに入っているNFTの画像を表示する

Pythonを利用してSolanaのNFT周りのデータを取得していく

- [Google Colab で実行するためのリンク](https://colab.research.google.com/github/regonn/Solana-NFT/blob/master/SolanaNFT_02.ipynb)

- [Softgateさんの NFT 技術基礎論](https://quirky-caution-3c5.notion.site/NFT-507d8038e2cd46de831e42ba2b4c0a90)

In [ ]:
!pip install solana -U

In [ ]:
# solana ライブラリで利用するもの
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.rpc.types import TokenAccountOpts

# エンコードされたデータやバイナリデータを扱うためのライブラリ
import base58
import base64
import struct

In [ ]:
# Solana上にデプロイされたプログラムID
METADATA_PROGRAM_ID = PublicKey('metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s')
TOKEN_PROGRAM_ID = PublicKey('TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA')

# 一つだけTokenを持っているWalletアドレス
WALLET_ADDRESS = "DDM479qxu1s9eZF8cf8ygRzSGUdhghNymdfdUTWJYxoT"

In [ ]:
# メインネットを利用する
client = Client("https://api.mainnet-beta.solana.com")

In [ ]:
# encoding= 'jsonParsed' を設定しないと、Base64 等エンコードされたdataが返ってくるので注意
opts = TokenAccountOpts(program_id = TOKEN_PROGRAM_ID, encoding= 'jsonParsed')

In [ ]:
# https://michaelhly.github.io/solana-py/api.html#solana.rpc.api.Client.get_token_accounts_by_owner
client_response = client.get_token_accounts_by_owner(PublicKey(WALLET_ADDRESS), opts=opts)

In [ ]:
client_response

In [ ]:
data = client_response['result']['value'][0]['account']['data']['parsed']

In [ ]:
data

In [ ]:
mint_address = data['info']['mint']

In [ ]:
mint_address

In [ ]:
# https://github.com/metaplex-foundation/python-api/blob/57499489443a1a31f5c77f4d66831aa06ef2eb4a/metaplex/metadata.py
# 上記のURLより取得

# metadataアカウントを取得する
def get_metadata_account(mint_key):
    return PublicKey.find_program_address(
        [b'metadata', bytes(METADATA_PROGRAM_ID), bytes(PublicKey(mint_key))],
        METADATA_PROGRAM_ID
    )[0]

# バイナリデータからmetadata情報を取り出す
def unpack_metadata_account(data):
    assert(data[0] == 4)
    i = 1
    source_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    mint_account = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
    i += 32
    name_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4
    name = struct.unpack('<' + "B"*name_len, data[i:i+name_len])
    i += name_len
    symbol_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    symbol = struct.unpack('<' + "B"*symbol_len, data[i:i+symbol_len])
    i += symbol_len
    uri_len = struct.unpack('<I', data[i:i+4])[0]
    i += 4 
    uri = struct.unpack('<' + "B"*uri_len, data[i:i+uri_len])
    i += uri_len
    fee = struct.unpack('<h', data[i:i+2])[0]
    i += 2
    has_creator = data[i] 
    i += 1
    creators = []
    verified = []
    share = []
    if has_creator:
        creator_len = struct.unpack('<I', data[i:i+4])[0]
        i += 4
        for _ in range(creator_len):
            creator = base58.b58encode(bytes(struct.unpack('<' + "B"*32, data[i:i+32])))
            creators.append(creator)
            i += 32
            verified.append(data[i])
            i += 1
            share.append(data[i])
            i += 1
    primary_sale_happened = bool(data[i])
    i += 1
    is_mutable = bool(data[i])
    metadata = {
        "update_authority": source_account,
        "mint": mint_account,
        "data": {
            "name": bytes(name).decode("utf-8").strip("\x00"),
            "symbol": bytes(symbol).decode("utf-8").strip("\x00"),
            "uri": bytes(uri).decode("utf-8").strip("\x00"),
            "seller_fee_basis_points": fee,
            "creators": creators,
            "verified": verified,
            "share": share,
        },
        "primary_sale_happened": primary_sale_happened,
        "is_mutable": is_mutable,
    }
    return metadata

In [ ]:
metadata_account = get_metadata_account(mint_address)

In [ ]:
metadata_account

In [ ]:
# https://michaelhly.github.io/solana-py/api.html#solana.rpc.api.Client.get_account_info
# get_account_info は data が json としてパースできない場合には base64 で返してくる
# 今回の場合もmetadataはバイナリデータなので、jsonParsed を指定しても base64 で渡ってくる

decoded_data = base64.b64decode(client.get_account_info(metadata_account)['result']['value']['data'][0])

In [ ]:
metadata = unpack_metadata_account(decoded_data)

In [ ]:
# オンチェーンメタデータ
metadata

In [ ]:
metadata_uri = metadata['data']['uri']

In [ ]:
# メタデータURL
metadata_uri

In [ ]:
import requests

In [ ]:
response = requests.get(metadata_uri)

In [ ]:
response.text

In [ ]:
response_json = response.json()

In [ ]:
# オフチェーンメタデータ
response_json

In [ ]:
from IPython.display import Image

In [ ]:
Image(url= response_json['image'])